In [4]:
# pip install nfl_data_py
import nfl_data_py as nfl
import pandas as pd

## Top NFL QB per year 2006-2023

In [40]:
import pandas as pd
array = [2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]
highestQBR = {}
for x in array:
    df = nfl.import_qbr([x])
    rank1 = df['rank'].idxmin()
    name = df.loc[rank1, 'name_display']
    highestQBR[x] = name

# highestQBR is a dict with the year and rank 1 QB
# incase this data is needed
print(highestQBR)

{2006: 'Peyton Manning', 2007: 'Tom Brady', 2008: 'Peyton Manning', 2009: 'Drew Brees', 2010: 'Tom Brady', 2011: 'Aaron Rodgers', 2012: 'Peyton Manning', 2013: 'Peyton Manning', 2014: 'Tony Romo', 2015: 'Carson Palmer', 2016: 'Matt Ryan', 2017: 'Carson Wentz', 2018: 'Patrick Mahomes', 2019: 'Lamar Jackson', 2020: 'Aaron Rodgers', 2021: 'Aaron Rodgers', 2022: 'Patrick Mahomes', 2023: 'Brock Purdy'}


## Top College QB Prospects 2006-2023

## Hypothesis Test?

| Anatomy of the hypothesis test |  Answer  |
|:------------------------------:|:--------:|
| Assumption                     | ... |
| Null hypothesis                | ... |
| Alternate hypothesis           | ... |
| Test statistic                 | ... |
| Rejection region shape         | ... |